In [1]:
!pip install datasets==2.10.1
#load_dataset sometimes hangs on a higher version
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.0 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 14.0 MB/s eta 0:00:00


# Model


In [2]:
from datasets import load_dataset

import torch
import torch.nn as nn
import numpy as np
import random
import numpy as np
from typing import List, Dict, Any

# we set up some seeds so that we can reproduce results
seed = 123
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

np.random.seed(seed)
random.seed(seed)
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True


In [3]:
# Loading data from json files using load_dataset
data_files = {"train": "all_train.json", "dev": "all_dev.json"}
dataset = load_dataset('json', data_files=data_files, chunksize=10<<23)

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating dev split: 0 examples [00:00, ? examples/s]

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-b77a65755d2ea510/0.0.0/0f7e3662623656454fcd2b650f34e886a7db4b9104504885bd462096cc7a9f51. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
from transformers import DistilBertTokenizer, DistilBertConfig, DistilBertPreTrainedModel, DistilBertModel
from torch.utils.data import DataLoader
from torch.nn.functional import softmax, nll_loss

# Initializing custom BERT model using github documentation at this link
# https://github.com/huggingface/transformers/blob/v4.35.0/src/transformers/models/distilbert/modeling_distilbert.py#L37
# This was required as the huggig face default models do not return a type paramater which is required
# by the model described in the paper for the purposes of calculation of the loss
class DistilBertQAmodel(DistilBertPreTrainedModel):
  def __init__(self, config):
    super().__init__(config)

    self.distilbert = DistilBertModel.from_pretrained("distilbert-base-uncased")
    self.qa_outputs = nn.Linear(config.dim, 2)
    self.answer_type = nn.Linear(config.dim, 2)

    self.dropout = nn.Dropout(config.qa_dropout)

    # Initialize weights and apply final processing
    self.post_init()


  def forward(self, input_ids, attention_mask, start_idx, end_idx, type_idx):

    distilbert_output = self.distilbert(input_ids=input_ids, attention_mask=attention_mask)
    hidden_states = distilbert_output[0] # (bs, max_query_len, dim)
    logits = self.qa_outputs(self.dropout(hidden_states))

    start_logits, end_logits = logits.split(1, dim=-1)
    start_logits = start_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
    end_logits = end_logits.squeeze(-1).contiguous()  # (bs, max_query_len)
    type_logits = self.answer_type(hidden_states[:, 0])


    # Calculating loss based on function descirbed in the paper

    # Softmaxing the probabilities obtained for the start positions, end positions, and answer type
    start_prob = softmax(start_logits, dim=1)
    end_prob = softmax(end_logits, dim=1)
    type_prob = softmax(type_logits, dim=1)

    # Using gather to select the probabilities of the correct indices
    start_loss = -torch.log(torch.gather(start_prob, 1, start_idx.unsqueeze(1)).squeeze(-1))
    end_loss = -torch.log(torch.gather(end_prob, 1, end_idx.unsqueeze(1)).squeeze(-1))
    type_loss = -torch.log(torch.gather(type_prob, 1, type_idx.unsqueeze(1)).squeeze(-1))

    # Taking the mean of the each of the losses calculated above. The loss calculated below corresponds to a single batch of data.
    loss = (start_loss + end_loss + type_loss).mean()

    # Returning the loss and logits from the forward pass
    return {'loss': loss, 'logits': (start_logits, end_logits, type_logits)}

# Loading the custom model created above using a pretrained DistilBertConfig
# Returning the loaded model, along with a DistilBertTokenizer
def load_model():
  config = DistilBertConfig.from_pretrained("distilbert-base-uncased")
  model = DistilBertQAmodel(config)
  tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
  return model, tokenizer

# Loading and returning data imported in the previous cell
def load_data():
  return dataset['train'], dataset['dev']

In [5]:
# Defining a child class to the PyTorch Dataset class so we can customize it for our data and
# configure the tokenizer created in the load_model() function
class QADataset(torch.utils.data.Dataset):
  """
  A PyTorch Dataset for our data that can be iterated through using __getitem__
  """
  def __init__(self, data_list, tokenizer : DistilBertTokenizer, max_length: int = 512) -> None:
    """
    Initializes the QADataset from a list of data, tokenizer, max_length of a sequence produced by the tokenizer

    """
    self.data_list = data_list
    self.tokenizer = tokenizer
    self.max_length = max_length

  def __len__(self) -> int:
    """
    Returns the number of questions/answers in the dataset.
    """
    return len(self.data_list)

  def __getitem__(self, index : int) -> Dict[str, Any]:
    """
    Retrieve a preprocessed data item from the dataset at the specified index.
    This is called when iterating through a QADataset
    """

    data = self.data_list[index]
    question = data['questions'][0]['input_text']
    context = data['contexts']
    answers = data['answers']

    # Tokenize context and question
    encodings = self.tokenizer.encode_plus(
        question,
        context,
        add_special_tokens=True,
        max_length=self.max_length,
        padding='max_length',
        truncation='only_second',
        return_attention_mask=True,
        return_overflowing_tokens=True,
        return_tensors='pt')

    # Initialize start and end positions
    start_positions = 0
    end_positions = 0

    # If there's a correct answer, set the start and end positions
    if data['has_correct_context'] and answers:
      # Find start and end of the answer in the context
      answer_start = answers[0]['span_start']
      answer_end = answers[0]['span_end']

      # Convert character start/end positions to token start/end positions
      start_positions = self.char_to_token_position(question, context, answer_start)
      end_positions = self.char_to_token_position(question, context, answer_end, end_pos=True)


    return {
        'input_ids': encodings['input_ids'][0],
        'attention_mask': encodings['attention_mask'][0],
        'start_positions': start_positions,
        'end_positions': end_positions,
        'true_type': 0 if (start_positions == 0 and end_positions == 0) else 1 # sets true_type to 0 if there is no answer and 1 if there is
    }

  def char_to_token_position(self, question, context, char_position, end_pos=False):
    """
    Converts a character position to a token position.
    """

    # Encode question to calculate start of context
    # Add 2 to length of question encoding to account for [CLS] before the question and [SEP] between question and context in tokenized data
    question_encoding = self.tokenizer.encode(question, add_special_tokens=False)
    context_start_pos = len(question_encoding) + 2

    # Encode context up to the character position and calculate token_position
    context_encoding = self.tokenizer.encode(context[:char_position], add_special_tokens=False)
    token_position = len(context_encoding) + context_start_pos

    if token_position > self.max_length - 1:
      return 0
    else:
      if end_pos:
        return token_position - 1
      return token_position

In [24]:
# Ignore Cell - used for testing and video explanation

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def char_to_token_position(question, context, char_position, end_pos=False):
    """
    Converts a character position to a token position.
    """

    # Encode question to calculate start of context
    # Add 2 to length of question encoding to account for [CLS] before the question and [SEP] between question and context in tokenized data
    question_encoding = tokenizer.encode(question, add_special_tokens=False)
    context_start_pos = len(question_encoding) + 2

    # Encode context up to the character position and calculate token_position
    context_encoding = tokenizer.encode(context[:char_position], add_special_tokens=False)
    token_position = len(context_encoding) + context_start_pos

    if token_position > 511:
      return 0
    else:
      if end_pos:
        return token_position - 1
      return token_position

dataset_qa = QADataset(dataset['dev'], tokenizer, 512)

data_sa = dataset['dev'][0]
encoding_sa = tokenizer.convert_ids_to_tokens(dataset_qa[0]['input_ids'])
print(data_sa)
start_token_sa = char_to_token_position(data_sa['questions'][0]['input_text'], data_sa['contexts'], data_sa['answers'][0]['span_start'])
end_token_sa = char_to_token_position(data_sa['questions'][0]['input_text'], data_sa['contexts'], data_sa['answers'][0]['span_end'], end_pos=True)
print("start: ", start_token_sa)
print("end: ", end_token_sa)
print("encoding: ", encoding_sa[start_token_sa:end_token_sa+1])
print("answer: ", data_sa['answers'][0]['span_text'])
print("answer character indices: ", data_sa['contexts'][data_sa['answers'][0]['span_start']:data_sa['answers'][0]['span_end']])


data_na = dataset['dev'][1200]
encoding_na = tokenizer.convert_ids_to_tokens(dataset_qa[1200]['input_ids'])
print(data_na)
start_token_na = 0
end_token_na = 0
print("start: ", start_token_na)
print("end: ", end_token_na)
print("encoding: ", encoding_na[start_token_na:end_token_na+1])
print("answer: ", data_na['answers'][0]['span_text'])

{'name': 'Mandalay Bay', 'id': '7811140318762480311', 'questions': [{'input_text': 'who is the owner of the mandalay bay in vegas'}], 'answers': [{'candidate_id': 0, 'input_text': 'short', 'span_end': 601, 'span_start': 576, 'span_text': 'MGM Resorts International'}], 'has_correct_context': True, 'contexts': "Mandalay Bay Location Paradise , Nevada , U.S. Address 3950 South Las Vegas Boulevard Opening date March 2 , 1999 ; 18 years ago ( March 2 , 1999 ) Theme Tropical No. of rooms 3,309 Total gaming space 135,000 sq ft ( 12,500 m ) Permanent shows Michael Jackson : One Signature attractions Mandalay Bay Convention Center Mandalay Bay Events Center Shark Reef House of Blues Mandalay Beach Notable restaurants Aureole Alain Ducasse Rivea Charlie Palmer Steak Fleur by Hubert Keller Kumi Lupo Red Square RM Seafood Stripsteak Border Grill Las Vegas Casino type Land - based Owner MGM Resorts International Renovated in 2002 , 2007 Coordinates 36 ° 5 ′ 30 '' N 115 ° 10 ′ 29 '' W \ufeff / \ufef

In [8]:
from torch.optim import Optimizer
from transformers import get_scheduler
from datasets import load_metric
from tqdm.auto import tqdm

def train_loop(model, train_data_loader, validation_data_loader, device, num_epochs):

  # Initializing optimizer and learning rate shcheduler to train model
  optimizer = torch.optim.AdamW(model.parameters(), lr=3e-5)
  lr_scheduler = get_scheduler(
    "linear",
    optimizer=optimizer,
    num_warmup_steps=50,
    num_training_steps=len(train_data_loader) * num_epochs
  )

  # Creating lists to store train and validation losses for each epoch of training
  train_losses = []
  validation_losses = []


  for epoch in range(num_epochs):
    # put the model in training mode (important that this is done each epoch,
    # since we put the model into eval mode during validation)
    model.train()

    # Create a progress bar to keep track of time/batches left to trian in each epoch
    print(f"Epoch {epoch + 1} training:")
    progress_bar_train = tqdm(enumerate(train_data_loader), total=len(train_data_loader))


    # Initialize the train and validation losses for the current epoch
    train_loss = 0
    validation_loss = 0

    # Iterating through each batch of data
    for batch_idx, batch in progress_bar_train:
      optimizer.zero_grad()

      # Get tokenized data for the current batch
      inputs = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      true_type = batch['true_type'].to(device)

      # Get outputs from forward pass of the model
      outputs = model(inputs, attention_mask, start_positions, end_positions, true_type)

      # Extract loss from model outputs and add it to train_loss
      loss = outputs['loss']
      train_loss += loss.item()

      # Backpropogate on loss
      loss.backward()

      optimizer.step()
      lr_scheduler.step()

      # Update progress bar
      progress_bar_train.update(1)

    # Switch the model to eval mode for validation
    model.eval()

    # Disabling gradient calculation to prevent memory erros
    with torch.no_grad():

      # Creating another progress bar for the the validation data
      progress_bar_validation = tqdm(enumerate(validation_data_loader), total=len(validation_data_loader))

      # Iterating through batches of the validation data
      for batch_idx, batch in progress_bar_validation:
        inputs = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        start_positions = batch['start_positions'].to(device)
        end_positions = batch['end_positions'].to(device)
        true_type = batch['true_type'].to(device)

        outputs = model(inputs, attention_mask, start_positions, end_positions, true_type)

        loss = outputs['loss']
        validation_loss += loss.item()
        progress_bar_validation.update(1)

      # Averaging the train and validation losses for the epoch over the length of data
      train_losses.append(train_loss/len(train_data_loader))
      validation_losses.append(validation_loss/len(validation_data_loader))

  return train_losses, validation_losses

In [9]:
def eval_loop(model, validation_data_loader, device):

  model.eval()

  progress_bar = tqdm(enumerate(validation_data_loader), total=len(validation_data_loader))

  # Creating lists to store precisions and recalls for each element of the validation dataset
  precision_list = []
  recall_list = []

  with torch.no_grad():

    for batch_idx, batch in progress_bar:
      inputs = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      true_type = batch['true_type'].to(device)

      output_logits = model(inputs, attention_mask, start_positions, end_positions, true_type)['logits']

      # Extracting the predicted start and ends for each answer in the batch
      pred_start = torch.argmax(output_logits[0], dim=1)
      pred_end = torch.argmax(output_logits[1], dim=1)

      # Interating over each element in the bacth
      for i in range(len(batch['input_ids'])):

        # Extracting true and precited answer token ids for the given element and converting to sets
        true_ans_tokens = set((inputs[i][start_positions[i]:end_positions[i]+1]).tolist())
        pred_ans_tokens = set((inputs[i][pred_start[i]:pred_end[i]+1]).tolist())

        # True positives are all the tokens that occur in the true and predicted spans
        true_positives = len(true_ans_tokens & pred_ans_tokens)
        # False positives are all the tokens that occur in the predicted span but not in the true span
        false_positives = len(pred_ans_tokens - true_ans_tokens)
        # False negatives are all the tokens that occur in the true span but not in the predicted span
        false_negatives = len(true_ans_tokens - pred_ans_tokens)

        # If the answer is a short answer and predicted is no answer or vice versa, set precision and recall to zero to avoid division by zero
        # When the answer is no answer and the model predicts no answer, this is also done as we want to measure precison and recall for short answers
        if true_positives == 0 and (false_positives == 0 or false_negatives == 0):
          precision = 0
          recall = 0
        else:
          precision = true_positives / (true_positives + false_positives)
          recall = true_positives / (true_positives + false_negatives)

        # Add precision and recall for the current element to the lists
        precision_list.append(precision)
        recall_list.append(recall)

      progress_bar.update(1)

  # Take the average precison and recall calculated over all elements and calculate f1_score using those values
  final_precision = sum(precision_list)/len(precision_list)
  final_recall = sum(recall_list)/len(recall_list)
  f1_score = (2 * final_precision * final_recall) / (final_precision + final_recall)

  return final_precision, final_recall, f1_score

In [10]:
# Another eval loop that uses a different method of calculation of precision and recall
def eval_loop2(model, validation_data_loader, device):

  model.eval()

  progress_bar = tqdm(enumerate(validation_data_loader), total=len(validation_data_loader))

  true_positives = 0
  false_positives = 0
  false_negatives = 0

  with torch.no_grad():

    for batch_idx, batch in progress_bar:
      inputs = batch['input_ids'].to(device)
      attention_mask = batch['attention_mask'].to(device)
      start_positions = batch['start_positions'].to(device)
      end_positions = batch['end_positions'].to(device)
      true_type = batch['true_type'].to(device)

      output_logits = model(inputs, attention_mask, start_positions, end_positions, true_type)['logits']

      pred_start = torch.argmax(output_logits[0], dim=1)
      pred_end = torch.argmax(output_logits[1], dim=1)

      for i in range(len(batch['input_ids'])):
        if true_type[i] == 1: # When ground truth is a short answer
          if pred_start[i] == start_positions[i] and pred_end[i] == end_positions[i]: # Short answer predicted is correct
            true_positives += 1
          elif pred_start[i] == 0 and pred_end[i] == 0: # Model predicts no answer when there is an answer
            false_negatives += 1
          else:
            false_positives += 1 # Model predicts incorrect answer span
        else: # When ground truth is no answer
          if pred_start[i] != 0 or pred_end[i] != 0: # Model predicts there is an answer when there is no answer in context
            false_negatives += 1

      progress_bar.update(1)

  print("True Positives: ", true_positives)
  print("False Positives: ", false_positives)
  print("False Negatives: ", false_negatives)

  # Calculate precision, recall, f1_score
  precision = true_positives / (true_positives + false_positives)
  recall = true_positives / (true_positives + false_negatives)
  f1_score = (2 * precision * recall) / (precision + recall)

  return precision, recall, f1_score

In [11]:
def main():
  device = "cuda" if torch.cuda.is_available() else "cpu"
  batch_size = 32

  # Load model and data
  model, tokenizer = load_model()
  model.to(device)
  train, validation = load_data()

  # Create QADataset data from the data produced by load_data() to input into PyTorch dataloader
  train_dataset = QADataset(train, tokenizer, 512)
  validation_dataset = QADataset(validation, tokenizer, 512)

  # Create train, validation dataloaders
  train_data_loader = DataLoader(train_dataset, batch_size, shuffle=True)
  validation_data_loader = DataLoader(validation_dataset, batch_size, shuffle=True)

  # Run train and eval loops
  train_losses, val_losses = train_loop(model, train_data_loader, validation_data_loader, device, num_epochs=2)
  print("TRAIN LOSSES: ", train_losses)
  print("VAL LOSSES: ", val_losses)
  precision, recall, f1_score  = eval_loop(model, validation_data_loader, device)

  # Printing final precision, recall and f1-score
  print("PRECISION: ", precision)
  print("RECALL: ", recall)
  print("F1-SCORE: ", f1_score)

if __name__ == "__main__":
  main()

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

  0%|          | 0/871 [00:00<?, ?it/s]

  0%|          | 0/55 [00:00<?, ?it/s]

TRAIN LOSSES:  [4.064317898142625, 2.305382804468223]
VAL LOSSES:  [2.815049013224515, 2.6478882897983897]


  0%|          | 0/55 [00:00<?, ?it/s]

PRECISION:  0.6946974406257216
RECALL:  0.7319428045390896
F1-SCORE:  0.7128339393495475
